In [1]:
import pandas as pd
import requests
import os
from zipfile import ZipFile, is_zipfile
import time
import json

# 1. Scrape data from censusreporter.org

In [2]:
## For each request, scrape data by one table on state
## For example, table B19001 is for Household Income in the Past 12 Months (In 2017 Inflation-adjusted Dollars), and each request scrapes one state
def get_data(table_code):
    url = ('https://api.censusreporter.org/1.0/data/download/latest?table_ids=%s&geo_ids=050|01000US&format=csv' % 
           (table_code))
    #print(url)
    filename = 'data/%s.zip' % (table_code) ## name datasets using (table_code)
    if not is_zipfile(filename): ## check if the file has been already scraped, if yes then avoid to do it again
        result = requests.get(url) ##key scrape
        if not os.path.isdir('data/%s' % table_code):
            os.makedirs('data/%s' % table_code)
        with open(filename, 'wb') as f:
            f.write(result.content) ## save zipfiles from result
    return is_zipfile(filename)

In [3]:
get_data('B15001') ## Sex by Age by Educational Attainment for the Population 18 Years and Over

True

In [5]:
get_data('B02001') ## Race

True

In [6]:
get_data('B992705') ## Allocation of Direct-purchase Health Insurance

True

In [7]:
get_data('B19001') ## Household Income in the Past 12 Months (In 2016 Inflation-adjusted Dollars

True

In [8]:
get_data('B11001') ## Household Type (Including Living Alone)

True

In [9]:
get_data('B01001') ## Sex by Age

True

In [10]:
get_data('B09001') ## Population Under 18 Years by Age

True

In [11]:
get_data('B20001') ## Sex by Earnings in the Past 12 Months (In 2016 Inflation-adjusted Dollars) for the Population 16 Years and Over With Earnings in the Past 12 Months

True

In [12]:
get_data('B18101') ## Sex by Age by Disability Status

True

In [13]:
get_data('B08136') ## Aggregate Travel Time to Work (In Minutes) of Workers by Means of Transportation to Work

True

In [14]:
get_data('B24011') ## Occupation by Median Earnings in the Past 12 Months (In 2016 Inflation-adjusted Dollars) for the Civilian Employed Population 16 Years and Over

True

In [15]:
get_data('B24124') ## Detailed Occupation for the Full-time, Year-round Civilian Employed Population 16 Years and Over

True

In [16]:
get_data('B23020') ## Mean Usual Hours Worked in the Past 12 Months for Workers 16 to 64 Years

True

In [17]:
get_data('B01003') ## Total Population

True

In [18]:
def open_csv(table_code):
    file_dir = 'data/'+table_code+'.zip'
    with ZipFile(file_dir) as f:
        files = f.namelist()
        ## save csv file
        csv = [f for f in files if f.lower().endswith('.csv')][0] ## extract csv file from zipfile
        df = pd.read_csv(f.open(csv)) 
        df['fips'] = df['geoid'].str.slice(7, 13) ## adding a column of 'fips'(zipode) from 'geoid'

        return df

In [19]:
sex_age_education = open_csv('B15001')
sex_age_education.shape

(3220, 169)

In [20]:
race = open_csv('B02001')
race.shape

(3220, 23)

In [21]:
sex_age = open_csv('B01001')
sex_age.shape

(3220, 101)

In [22]:
children = open_csv('B09001')
children.shape

(3220, 23)

In [23]:
family = open_csv('B11001')
family.shape

(3220, 21)

In [24]:
sex_age_disability = open_csv('B18101')
sex_age_disability.shape

(3220, 81)

In [25]:
household_income = open_csv('B19001')
household_income.shape

(3220, 37)

In [26]:
sex_earnings = open_csv('B20001')
sex_earnings.shape

(3220, 89)

In [27]:
travel_time = open_csv('B08136')
travel_time.shape

(3220, 27)

In [28]:
detail_occupation = open_csv('B24124')
detail_occupation.shape

(3220, 1055)

In [29]:
hours_worked = open_csv('B23020')
hours_worked.shape

(3220, 11)

In [30]:
population = open_csv('B01003')
population.shape

(3220, 5)

In [31]:
health_insurance = open_csv('B992705')
health_insurance.shape

(3220, 9)

## Transfer json file to dictionary from one zipfile

In [32]:
def get_dictionary(table_code):
    file_dir = 'data/'+table_code+'.zip'
    with ZipFile(file_dir) as zipfile: 
        files = zipfile.namelist()
        ## save csv file
        json_file = [f for f in files if f.lower().endswith('.json')][0] ##get json file from zipfile
        f = zipfile.open(json_file)
        j = json.load(f)
        table_data = list(j['tables'].values())[0]
        tname = table_data['title']
        column_dic = table_data['columns']
        columns = []
        name_stack = []
        for c in sorted(column_dic.keys()):
            record = column_dic[c]
            name = record['name']
            indent = record['indent']
            if indent == 0:
                columns.append((c, name))
            else:
                if not name_stack or indent > name_stack[-1][0]:
                    name_stack.append((indent, name))
                if indent == name_stack[-1][0]:
                    name_stack[-1] = (indent, name)
                if indent < name_stack[-1][0]:
                    name_stack = [x for x in name_stack if x[0] < indent]
                    name_stack.append((indent, name))
                columns.append((c, ' '.join([x[1] for x in name_stack])))
        df = pd.DataFrame(columns, columns=['columns', 'description'])
        df['table'] = tname
        return df

In [33]:
get_dictionary('B01003')

,columns,description,table
0,B01003001,Total,Total Population


## Combile all dictionary together in one file

In [34]:
table_list = [f.replace('.zip','') for f in os.listdir('data') if f.endswith('.zip')]
census_dictionary = pd.concat([
    get_dictionary(table_code) for table_code in table_list
])

In [35]:
census_dictionary.sample(10)

,columns,description,table
28,B18101029,Female: 18 to 34 years: With a disability,Sex by Age by Disability Status
229,B24124230,"Food servers, nonrestaurant","Detailed Occupation for the Full-time, Year-ro..."
207,B24124208,Firefighters,"Detailed Occupation for the Full-time, Year-ro..."
80,B15001081,Female: 65 years and over: Associate's degree,Sex by Age by Educational Attainment for the P...
4,B24011005,"Management, business, science, and arts occupa...",Occupation by Median Earnings in the Past 12 M...
2,B23020002,Male,Mean Usual Hours Worked in the Past 12 Months ...
78,B15001079,Female: 65 years and over: High school graduat...,Sex by Age by Educational Attainment for the P...
36,B20001037,"Female: $40,000 to $44,999",Sex by Earnings in the Past 12 Months (In 2017...
342,B24124343,"Carpet, floor, and tile installers and finishers","Detailed Occupation for the Full-time, Year-ro..."
12,B24124013,Training and development managers,"Detailed Occupation for the Full-time, Year-ro..."


In [36]:
census_dictionary.to_csv('data/dictionary.csv', index=False)

## preprocess one table to calculate ratio for all class

In [37]:
def process_table(df, total_col_name, doordash_col='avg_subtotal'):
    df1 = df.copy()
    df1 = df1.loc[~df1['name'].duplicated(), :]
    cols = [c for c in df1.columns if 'Error' not in c and ##
            c not in ('fips', 'geoid', 'name', total_col_name)]
    to_drop = []
    if total_col_name:
        total_col = df1[total_col_name]
        for c in cols:
            df1[c] /= total_col ## calculate ratio by dividing total
        df1.loc[total_col == 0, cols] = 0
    else:
        for c in cols:
            if df1[c].max() > 0:
                df1[c] /= df1[c].max() ##normalize
            else:
                to_drop.append(c)
    df1 = df1.loc[:, ['fips'] + cols]
    df1.drop(to_drop, axis=1, inplace=True)
    # df1['fips'] = df1['fips'].astype(str).str.zfill(5)
    return df1.dropna()

In [38]:
census_preprocessed = process_table(sex_age_education, 'B15001001').merge(
    process_table(race, 'B02001001')).merge(
    process_table(health_insurance, 'B992705001')).merge(
    process_table(family, 'B11001001')).merge(
    process_table(children, 'B09001001')).merge(
    process_table(sex_age, 'B01001001')).merge(
    process_table(household_income, 'B19001001')).merge(
    process_table(sex_age_disability, 'B18101001')).merge(
    process_table(detail_occupation, 'B24124001')).merge(
    process_table(hours_worked, None)).merge(
    process_table(population, None))

#family = process_census('../../census_data/data/family.csv', 'B11001001', doordash_col=dda_col)
#transportation = process_census('../../census_data/data/sex_transportaion.csv', 'B08006001', doordash_col=dda_col)
#children = process_census('../../census_data/data/children.csv', 'B09001001', doordash_col=dda_col)
#travel_time =  process_census('../../census_data/data/travel_time.csv', None, doordash_col=dda_col)
#sex_age = process_census('../../census_data/data/sex_age.csv', 'B01001001', doordash_col=dda_col)
#household_income = process_census('../../census_data/data/household_income.csv', 'B19001001', doordash_col=dda_col)
#sex_disability = process_census('../../census_data/data/sex_age_disability.csv', 'B18101001', doordash_col=dda_col)
#occupation_earning = process_census('../../census_data/data/occupation_earning.csv', 'B24011001', doordash_col=dda_col)
census_preprocessed.shape

(3220, 742)

In [39]:
census_preprocessed = census_preprocessed.loc[census_preprocessed.max(axis=1) > 0,
                                             census_preprocessed.max(axis=0) > 0] ## exclude columns and rows with all 0
census_preprocessed.shape

(3220, 217)

In [40]:
census_preprocessed.sample(10)

,fips,B15001002,B15001003,B15001004,B15001005,B15001006,B15001007,B15001008,B15001009,B15001010,...,B18101034,B18101035,B18101036,B18101037,B18101038,B18101039,B23020001,B23020002,B23020003,B01003001
203,06035,0.686368,0.119531,0.005067,0.037739,0.051579,0.020684,0.003063,0.001172,0.000227,...,0.051934,0.014398,0.037536,0.037398,0.021160,0.016238,0.626409,0.619549,0.666667,0.003114
232,06093,0.495636,0.045773,0.000317,0.005185,0.021806,0.015786,0.002189,0.000173,0.000317,...,0.071103,0.014861,0.056242,0.048293,0.023824,0.024469,0.594203,0.607519,0.620112,0.004307
1239,26017,0.485695,0.053456,0.000564,0.007434,0.018802,0.020529,0.003537,0.002374,0.000216,...,0.055438,0.012681,0.042757,0.047613,0.022106,0.025506,0.607085,0.604511,0.651769,0.010425
1001,21017,0.479167,0.057971,0.003170,0.009058,0.026850,0.013522,0.001488,0.001747,0.002135,...,0.054812,0.014701,0.040111,0.043799,0.020662,0.023137,0.634461,0.642105,0.674115,0.001981
2242,41069,0.471033,0.038623,0.000000,0.002519,0.011755,0.010076,0.006717,0.007557,0.000000,...,0.082153,0.019122,0.063031,0.094901,0.033286,0.061615,0.626409,0.663158,0.614525,0.000140
1838,36021,0.495779,0.048640,0.001872,0.006132,0.015570,0.013041,0.005396,0.006530,0.000100,...,0.063659,0.013909,0.049750,0.047995,0.024474,0.023521,0.624799,0.612030,0.683426,0.006084
491,13213,0.489287,0.059319,0.001188,0.012360,0.023225,0.019592,0.002411,0.000543,0.000000,...,0.045585,0.016384,0.029201,0.027188,0.013811,0.013378,0.634461,0.619549,0.689013,0.003903
2819,50027,0.485354,0.040740,0.001993,0.009521,0.016296,0.008879,0.000908,0.003144,0.000000,...,0.064773,0.015572,0.049201,0.047081,0.021640,0.025441,0.621578,0.616541,0.672253,0.005490
2351,45071,0.481316,0.066064,0.001321,0.013348,0.027306,0.022089,0.000644,0.001355,0.000000,...,0.058408,0.012060,0.046347,0.044720,0.026122,0.018598,0.615137,0.606015,0.672253,0.003752
136,05051,0.474381,0.047672,0.000885,0.008801,0.020629,0.014509,0.001719,0.000924,0.000205,...,0.068237,0.016553,0.051684,0.054846,0.028921,0.025925,0.628019,0.621053,0.679702,0.009697


In [41]:
census_preprocessed.to_csv('data/cencus_preprocessed.csv', index=False)